In [ ]:
import os
import json
import shutil
import random
import numpy as np
from tqdm import tqdm
import kagglehub
from pathlib import Path
import yaml

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

# Download dataset if not already downloaded
def download_dataset():
    print("Downloading supermarket shelves dataset...")
    path = kagglehub.dataset_download("humansintheloop/supermarket-shelves-dataset")
    print(f"Dataset downloaded to: {path}")

    # Explore the directory structure
    print("\nExploring dataset directory structure:")
    found_image_dir = None
    found_anno_dir = None

    for root, dirs, files in os.walk(path):
        # Skip .git directories
        if '.git' in root:
            continue

        level = root.replace(path, '').count(os.sep)
        indent = ' ' * 4 * level
        print(f"{indent}{os.path.basename(root)}/")

        # Look for likely image and annotation directories
        image_exts = ['.jpg', '.jpeg', '.png']
        json_exts = ['.json']

        # Count image and JSON files in this directory
        img_count = sum(1 for f in files if os.path.splitext(f)[1].lower() in image_exts)
        json_count = sum(1 for f in files if os.path.splitext(f)[1].lower() in json_exts)

        # Print file counts
        if img_count > 0:
            print(f"{indent}    (Found {img_count} image files)")
            # Record this as a potential image directory if it has the most images so far
            if found_image_dir is None or img_count > found_image_dir[1]:
                found_image_dir = (root, img_count)

        if json_count > 0:
            print(f"{indent}    (Found {json_count} JSON files)")
            # Record this as a potential annotation directory if it has the most JSONs so far
            if found_anno_dir is None or json_count > found_anno_dir[1]:
                found_anno_dir = (root, json_count)

    # Print the discovered directories
    if found_image_dir:
        print(f"\nFound likely image directory: {found_image_dir[0]} ({found_image_dir[1]} images)")
    if found_anno_dir:
        print(f"Found likely annotation directory: {found_anno_dir[0]} ({found_anno_dir[1]} JSON files)")

    # Create a dataset info dictionary to return
    dataset_info = {
        'base_path': path,
        'image_dir': found_image_dir[0] if found_image_dir else None,
        'anno_dir': found_anno_dir[0] if found_anno_dir else None,
        'meta_dir': found_anno_dir[0] if found_anno_dir else path  # Assume meta.json is in the same dir as annotations
    }

    return dataset_info

# Function to read meta.json and get class information
def get_class_info(dataset_info):
    # Look for meta.json in various locations
    possible_meta_paths = [
        os.path.join(dataset_info['meta_dir'], "meta.json"),
        os.path.join(dataset_info['base_path'], "meta.json")
    ]

    # Also look in subdirectories of the base path
    for root, _, files in os.walk(dataset_info['base_path']):
        if "meta.json" in files:
            possible_meta_paths.append(os.path.join(root, "meta.json"))

    meta_path = None
    meta_data = None

    # Try each path until we find a valid meta.json
    for path in possible_meta_paths:
        if os.path.exists(path):
            print(f"Found meta.json at {path}")
            try:
                with open(path, 'r') as f:
                    meta_data = json.load(f)
                meta_path = path
                break
            except Exception as e:
                print(f"Error reading {path}: {e}")

    if not meta_data:
        print("Could not find or parse meta.json, checking annotation files instead")
        return infer_classes_from_annotations(dataset_info)

    # Extract classes - structure might vary, so let's try common patterns
    classes = []
    if "classes" in meta_data:
        # Handle if classes are dictionaries with 'title' field
        raw_classes = meta_data["classes"]
        if isinstance(raw_classes, list) and len(raw_classes) > 0 and isinstance(raw_classes[0], dict):
            if "title" in raw_classes[0]:
                classes = [cls["title"] for cls in raw_classes]
            elif "name" in raw_classes[0]:
                classes = [cls["name"] for cls in raw_classes]
            else:
                # If we can't find title or name, use the first string key we find
                for key in raw_classes[0].keys():
                    if isinstance(raw_classes[0][key], str):
                        classes = [cls[key] for cls in raw_classes]
                        break
        else:
            classes = raw_classes
    elif "categories" in meta_data:
        classes = [cat["name"] for cat in meta_data["categories"]]
    elif "labels" in meta_data:
        classes = meta_data["labels"]

    # If still empty, try to infer from an annotation file
    if not classes:
        print("Classes not found in meta.json, inferring from annotation file...")
        return infer_classes_from_annotations(dataset_info)

    print(f"Found {len(classes)} classes: {classes}")
    return classes

# Infer classes from annotations as a fallback
def infer_classes_from_annotations(dataset_info):
    print("Attempting to infer classes from annotation files...")
    anno_dir = dataset_info['anno_dir']

    if not anno_dir or not os.path.exists(anno_dir):
        print(f"Error: Annotation directory not found at {anno_dir}")
        return []

    classes = set()

    # Get all JSON files in the annotation directory
    anno_files = [f for f in os.listdir(anno_dir) if f.endswith('.json')]

    # Limit to the first 10 files to avoid long processing time
    anno_files = anno_files[:10]

    for anno_file in anno_files:
        try:
            with open(os.path.join(anno_dir, anno_file), 'r') as f:
                anno_data = json.load(f)

            # SuperviselyIO format
            if "objects" in anno_data:
                for obj in anno_data["objects"]:
                    if "classTitle" in obj:
                        classes.add(obj["classTitle"])

            # LabelMe format
            elif "shapes" in anno_data:
                for shape in anno_data["shapes"]:
                    if "label" in shape:
                        classes.add(shape["label"])

            # COCO format
            elif "annotations" in anno_data:
                if "categories" in anno_data:
                    for cat in anno_data["categories"]:
                        if "name" in cat:
                            classes.add(cat["name"])

        except Exception as e:
            print(f"Error processing {anno_file}: {e}")

    classes = list(classes)
    print(f"Inferred {len(classes)} classes: {classes}")
    return classes

# Convert COCO/JSON annotations to YOLO format
def convert_annotations(dataset_info, classes):
    images_path = dataset_info['image_dir']
    annotations_path = dataset_info['anno_dir']

    if not images_path or not os.path.exists(images_path):
        raise FileNotFoundError(f"Image directory not found at {images_path}")

    if not annotations_path or not os.path.exists(annotations_path):
        raise FileNotFoundError(f"Annotation directory not found at {annotations_path}")

    print(f"Using image directory: {images_path}")
    print(f"Using annotation directory: {annotations_path}")

    # Create output directories
    output_base = "yolo_dataset"
    os.makedirs(os.path.join(output_base, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_base, "labels"), exist_ok=True)

    # Create class mapping
    class_to_id = {cls_name: i for i, cls_name in enumerate(classes)}

    # Store paths for all processed files
    all_image_paths = []

    print("Converting annotations to YOLO format...")

    # Get all image files
    image_files = [f for f in os.listdir(images_path)
                  if f.lower().endswith(('.jpg', '.jpeg', '.png'))]

    # Get all annotation files
    anno_files = [f for f in os.listdir(annotations_path)
                 if f.lower().endswith('.json')]

    print(f"Found {len(image_files)} images and {len(anno_files)} annotation files")

    # Create a mapping from image names to annotation files
    # This handles cases where the naming conventions might differ
    image_to_anno = {}

    for anno_file in anno_files:
        # Try different matching strategies
        base_name = os.path.splitext(anno_file)[0]

        # Case 1: annotation is named exactly like the image with .json added
        if base_name in image_files:
            image_to_anno[base_name] = anno_file

        # Case 2: annotation is named like the image without extension
        for img_file in image_files:
            img_base = os.path.splitext(img_file)[0]
            if img_base == base_name:
                image_to_anno[img_file] = anno_file
                break

    print(f"Successfully matched {len(image_to_anno)} images with annotations")

    for img_file, anno_file in tqdm(image_to_anno.items()):
        img_path = os.path.join(images_path, img_file)
        anno_path = os.path.join(annotations_path, anno_file)

        try:
            with open(anno_path, 'r') as f:
                anno_data = json.load(f)

            # Get image dimensions - try to handle different annotation formats
            img_width = img_height = None

            # Try to get dimensions from the annotation file
            if "imageWidth" in anno_data and "imageHeight" in anno_data:
                img_width = anno_data["imageWidth"]
                img_height = anno_data["imageHeight"]
            elif "width" in anno_data and "height" in anno_data:
                img_width = anno_data["width"]
                img_height = anno_data["height"]
            elif "image" in anno_data and isinstance(anno_data["image"], dict):
                img_width = anno_data["image"].get("width")
                img_height = anno_data["image"].get("height")

            # If dimensions not found, use pillow to get them
            if not img_width or not img_height:
                try:
                    from PIL import Image
                    with Image.open(img_path) as img:
                        img_width, img_height = img.size
                except Exception as e:
                    print(f"Error getting image dimensions for {img_file}: {e}")
                    continue

            # Extract annotations
            yolo_annotations = []

            # Handle SuperviselyIO format which seems to be used in this dataset
            if "objects" in anno_data:
                for obj in anno_data["objects"]:
                    # Get class title
                    class_title = obj.get("classTitle")

                    # Skip if class not in our class list
                    if class_title not in class_to_id:
                        print(f"Warning: Class '{class_title}' not in class list. Available classes: {classes}")
                        continue

                    class_id = class_to_id[class_title]

                    # Get bounding box points
                    if "points" in obj:
                        # Extract points - format can vary
                        points = None

                        # Handle different points formats
                        if "exterior" in obj["points"]:
                            points = obj["points"]["exterior"]
                        elif isinstance(obj["points"], list):
                            points = obj["points"]

                        if points and len(points) >= 2:
                            # For rectangle, find min/max points
                            x_coords = [p[0] for p in points]
                            y_coords = [p[1] for p in points]

                            x1, y1 = min(x_coords), min(y_coords)
                            x2, y2 = max(x_coords), max(y_coords)

                            # Calculate width and height
                            w = x2 - x1
                            h = y2 - y1

                            # Convert to YOLO format
                            x_center = (x1 + w/2) / img_width
                            y_center = (y1 + h/2) / img_height
                            w_norm = w / img_width
                            h_norm = h / img_height

                            yolo_annotations.append(f"{class_id} {x_center} {y_center} {w_norm} {h_norm}")

                    # Handle bitmap format by converting to bounding box
                    elif "bitmap" in obj:
                        # Implement if needed for this dataset
                        continue

                    # Handle polygon format
                    elif "polygon" in obj:
                        # Implement if needed
                        continue

            # Handle LabelMe format as alternative
            elif "shapes" in anno_data:
                for shape in anno_data["shapes"]:
                    label = shape.get("label")
                    if label not in class_to_id:
                        continue

                    class_id = class_to_id[label]
                    points = shape.get("points", [])

                    if shape.get("shape_type") == "rectangle" and len(points) == 2:
                        # Rectangle format: [[x1,y1], [x2,y2]]
                        x1, y1 = points[0]
                        x2, y2 = points[1]

                        # Calculate width and height
                        x = min(x1, x2)
                        y = min(y1, y2)
                        w = abs(x2 - x1)
                        h = abs(y2 - y1)

                        # Convert to YOLO format
                        x_center = (x + w/2) / img_width
                        y_center = (y + h/2) / img_height
                        w_norm = w / img_width
                        h_norm = h / img_height

                        yolo_annotations.append(f"{class_id} {x_center} {y_center} {w_norm} {h_norm}")

                    elif shape.get("shape_type") == "polygon" and len(points) > 2:
                        # Get bounding box from polygon
                        x_coords = [p[0] for p in points]
                        y_coords = [p[1] for p in points]

                        x1, y1 = min(x_coords), min(y_coords)
                        x2, y2 = max(x_coords), max(y_coords)

                        # Calculate width and height
                        w = x2 - x1
                        h = y2 - y1

                        # Convert to YOLO format
                        x_center = (x1 + w/2) / img_width
                        y_center = (y1 + h/2) / img_height
                        w_norm = w / img_width
                        h_norm = h / img_height

                        yolo_annotations.append(f"{class_id} {x_center} {y_center} {w_norm} {h_norm}")

            # Save YOLO annotations to file
            if yolo_annotations:
                all_image_paths.append(img_path)

                # Create label file with same name as image but .txt extension
                label_filename = os.path.splitext(img_file)[0] + ".txt"
                label_path = os.path.join(output_base, "labels", label_filename)

                # Write annotations to file
                with open(label_path, 'w') as f:
                    f.write('\n'.join(yolo_annotations))

                # Copy image to output directory
                output_img_path = os.path.join(output_base, "images", img_file)
                shutil.copy(img_path, output_img_path)

        except Exception as e:
            print(f"Error processing {anno_path}: {e}")

    print(f"Successfully processed {len(all_image_paths)} images")
    return all_image_paths, output_base

# Split data into train and validation sets
def split_data(all_image_paths, output_base, train_ratio=0.8):
    random.shuffle(all_image_paths)
    split_idx = int(len(all_image_paths) * train_ratio)

    train_images = all_image_paths[:split_idx]
    val_images = all_image_paths[split_idx:]

    print(f"Split data: {len(train_images)} training images, {len(val_images)} validation images")

    # Create train/val directories
    train_img_dir = os.path.join(output_base, "train", "images")
    train_label_dir = os.path.join(output_base, "train", "labels")
    val_img_dir = os.path.join(output_base, "val", "images")
    val_label_dir = os.path.join(output_base, "val", "labels")

    os.makedirs(train_img_dir, exist_ok=True)
    os.makedirs(train_label_dir, exist_ok=True)
    os.makedirs(val_img_dir, exist_ok=True)
    os.makedirs(val_label_dir, exist_ok=True)

    # Copy files to respective directories
    for img_path in train_images:
        img_filename = os.path.basename(img_path)
        label_filename = os.path.splitext(img_filename)[0] + ".txt"

        # Copy image
        shutil.copy(img_path, os.path.join(train_img_dir, img_filename))

        # Copy label if it exists
        src_label = os.path.join(output_base, "labels", label_filename)
        if os.path.exists(src_label):
            shutil.copy(src_label, os.path.join(train_label_dir, label_filename))

    for img_path in val_images:
        img_filename = os.path.basename(img_path)
        label_filename = os.path.splitext(img_filename)[0] + ".txt"

        # Copy image
        shutil.copy(img_path, os.path.join(val_img_dir, img_filename))

        # Copy label if it exists
        src_label = os.path.join(output_base, "labels", label_filename)
        if os.path.exists(src_label):
            shutil.copy(src_label, os.path.join(val_label_dir, label_filename))

    return train_img_dir, val_img_dir

# Create YAML configuration file for YOLOv8
def create_yaml_config(output_base, classes):
    config = {
        'train': os.path.join(output_base, 'train', 'images'),
        'val': os.path.join(output_base, 'val', 'images'),
        'nc': len(classes),
        'names': classes
    }

    yaml_path = os.path.join(output_base, 'dataset.yaml')
    with open(yaml_path, 'w') as f:
        yaml.dump(config, f, sort_keys=False)

    print(f"Created YOLOv8 config at {yaml_path}")
    return yaml_path

# Main function to run the data preparation and training
def main():
    try:
        # 1. Download dataset and explore directory structure
        dataset_info = download_dataset()

        # 2. Get class information
        classes_data = get_class_info(dataset_info)

        # Handle classes that are dictionaries
        if classes_data and isinstance(classes_data, list):
            if classes_data and isinstance(classes_data[0], dict):
                # Extract the 'title' field as class names
                if "title" in classes_data[0]:
                    classes = [cls["title"] for cls in classes_data]
                elif "name" in classes_data[0]:
                    classes = [cls["name"] for cls in classes_data]
                else:
                    print("Warning: Could not determine class names from dictionaries.")
                    classes = [f"class_{i}" for i in range(len(classes_data))]
            else:
                classes = classes_data
        else:
            classes = classes_data

        print(f"Using classes: {classes}")

        if not classes:
            print("Error: No classes found for the dataset.")
            return

        # 3. Convert annotations to YOLO format
        all_image_paths, output_base = convert_annotations(dataset_info, classes)

        if not all_image_paths:
            print("Error: No valid image-annotation pairs found.")
            return

        # 4. Split data into train and validation sets
        split_data(all_image_paths, output_base)

        # 5. Create YAML configuration
        yaml_path = create_yaml_config(output_base, classes)

        # 6. Install YOLOv8 if not already installed
        try:
            import ultralytics
            print("Ultralytics already installed")
        except ImportError:
            print("Installing ultralytics...")
            !pip install -q ultralytics

        # 7. Train YOLOv8 model
        print("\nSetup complete! Now you can train the model with:")
        print("from ultralytics import YOLO")
        print("model = YOLO('yolov8n.pt')  # Load a pretrained YOLOv8 model")
        print(f"results = model.train(data='{yaml_path}', epochs=100, imgsz=640, batch=16, name='supermarket_model')")

    except Exception as e:
        print(f"An error occurred: {e}")
        import traceback
        traceback.print_exc()  # Print the full error traceback for debugging

if __name__ == "__main__":
    main()

100%|██████████| 131M/131M [00:00<00:00, 159MB/s]

Extracting files...


Dataset downloaded to: /root/.cache/kagglehub/datasets/humansintheloop/supermarket-shelves-dataset/versions/2

Exploring dataset directory structure:
2/
    Supermarket shelves/
        (Found 1 JSON files)
        Supermarket shelves/
            annotations/
                (Found 45 JSON files)
            images/
                (Found 45 image files)

Found likely image directory: /root/.cache/kagglehub/datasets/humansintheloop/supermarket-shelves-dataset/versions/2/Supermarket shelves/Supermarket shelves/images (45 images)
Found likely annotation directory: /root/.cache/kagglehub/datasets/humansintheloop/supermarket-shelves-dataset/versions/2/Supermarket shelves/Supermarket shelves/annotations (45 JSON files)
Found meta.json at /root/.cache/kagglehub/datasets/humansintheloop/supermarket-shelves-dataset/versions/2/Supermarket shelves/meta.json
Found 2 classes: ['Price', 'Product']
Using classes: ['Price', 'Product']
Using image directory: /root/.cache/kagglehub/datasets/humansinth

100%|██████████| 45/45 [00:00<00:00, 84.61it/s]


Successfully processed 45 images
Split data: 36 training images, 9 validation images
Created YOLOv8 config at yolo_dataset/dataset.yaml
Installing ultralytics...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
import os
import yaml
import shutil
from pathlib import Path

# 1. Check the current directory structure
print("Current working directory:", os.getcwd())
yolo_dataset_dir = "yolo_dataset"

# Check if directories exist
train_dir = os.path.join(yolo_dataset_dir, "train", "images")
val_dir = os.path.join(yolo_dataset_dir, "val", "images")
yaml_path = os.path.join(yolo_dataset_dir, "dataset.yaml")

print(f"Train directory exists: {os.path.exists(train_dir)}")
print(f"Validation directory exists: {os.path.exists(val_dir)}")
print(f"YAML file exists: {os.path.exists(yaml_path)}")

# 2. Fix the YAML file to use absolute paths
if os.path.exists(yaml_path):
    with open(yaml_path, 'r') as f:
        data = yaml.safe_load(f)

    print("Current YAML content:", data)

    # Fix paths to be absolute
    current_dir = os.getcwd()
    data['train'] = os.path.abspath(train_dir)
    data['val'] = os.path.abspath(val_dir)

    # Write back the updated YAML
    with open(yaml_path, 'w') as f:
        yaml.dump(data, f, sort_keys=False)

    print("Updated YAML content:", data)
    print(f"Updated YAML saved to {yaml_path}")

# 3. Alternative: Create a new YAML file with correct paths
new_yaml_path = "dataset_fixed.yaml"
with open(new_yaml_path, 'w') as f:
    yaml_data = {
        'train': os.path.abspath(train_dir),
        'val': os.path.abspath(val_dir),
        'nc': 2,  # Number of classes, change if different
        'names': ['Price', 'Product']  # Class names, update if different
    }
    yaml.dump(yaml_data, f, sort_keys=False)

print(f"Created new YAML file at {new_yaml_path} with absolute paths")
print(f"Use this command to train:\n")
print(f"from ultralytics import YOLO")
print(f"model = YOLO('yolov8n.pt')")
print(f"results = model.train(data='{new_yaml_path}', epochs=100, imgsz=640, batch=16, name='supermarket_model')")

Current working directory: /content
Train directory exists: True
Validation directory exists: True
YAML file exists: True
Current YAML content: {'train': 'yolo_dataset/train/images', 'val': 'yolo_dataset/val/images', 'nc': 2, 'names': ['Price', 'Product']}
Updated YAML content: {'train': '/content/yolo_dataset/train/images', 'val': '/content/yolo_dataset/val/images', 'nc': 2, 'names': ['Price', 'Product']}
Updated YAML saved to yolo_dataset/dataset.yaml
Created new YAML file at dataset_fixed.yaml with absolute paths
Use this command to train:

from ultralytics import YOLO
model = YOLO('yolov8n.pt')
results = model.train(data='dataset_fixed.yaml', epochs=100, imgsz=640, batch=16, name='supermarket_model')


In [ ]:
from ultralytics import YOLO
model = YOLO('yolov8n.pt')
results = model.train(data='dataset_fixed.yaml', epochs=60, imgsz=640, batch=16, name='supermarket_model')

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 6.25M/6.25M [00:00<00:00, 103MB/s]


Ultralytics 8.3.140 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=dataset_fixed.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=60, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=supermarket_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrain

100%|██████████| 755k/755k [00:00<00:00, 25.3MB/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

      60/60      6.49G      1.462      1.124      1.013        387        640: 100%|██████████| 3/3 [00:00<00:00,  3.03it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  2.57it/s]

                   all          9       2129      0.445      0.344      0.378      0.205



60 epochs completed in 0.061 hours.
Optimizer stripped from runs/detect/supermarket_model/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/supermarket_model/weights/best.pt, 6.2MB

Validating runs/detect/supermarket_model/weights/best.pt...
Ultralytics 8.3.140 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:00<00:00,  1.34it/s]


                   all          9       2129      0.444      0.344      0.379      0.205
                 Price          9        362      0.548      0.282      0.408      0.241
               Product          9       1767      0.341      0.407      0.349      0.169
Speed: 0.6ms preprocess, 4.6ms inference, 0.0ms loss, 7.5ms postprocess per image
Results saved to runs/detect/supermarket_model


In [ ]:
from google.colab import files

# Download the best model
files.download('runs/detect/supermarket_model/weights/best.pt')

# Optional: Also download the training results
files.download('runs/detect/supermarket_model/results.csv')  # Training metrics
files.download('runs/detect/supermarket_model/confusion_matrix.png')  # Confusion matrix

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
from ultralytics import YOLO

# Load your trained model
model = YOLO('runs/detect/supermarket_model/weights/best.pt')

# Export to ONNX format (for deployment)
model.export(format='onnx')

# Other formats available: 'torchscript', 'tflite', 'openvino', etc.
# For example: model.export(format='tflite')

Ultralytics 8.3.140 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
💡 ProTip: Export to OpenVINO format for best performance on Intel CPUs. Learn more at https://docs.ultralytics.com/integrations/openvino/
Model summary (fused): 72 layers, 3,006,038 parameters, 0 gradients, 8.1 GFLOPs

PyTorch: starting from 'runs/detect/supermarket_model/weights/best.pt' with input shape (1, 3, 640, 640) BCHW and output shape(s) (1, 6, 8400) (6.0 MB)
requirements: Ultralytics requirements ['onnx>=1.12.0,<1.18.0', 'onnxslim>=0.1.53', 'onnxruntime-gpu'] not found, attempting AutoUpdate...

requirements: AutoUpdate success ✅ 6.3s
WARNING ⚠️ requirements: Restart runtime or rerun command for updates to take effect


ONNX: starting export with onnx 1.17.0 opset 19...
ONNX: slimming with onnxslim 0.1.53...
ONNX: export success ✅ 7.7s, saved as 'runs/detect/supermarket_model/weights/best.onnx' (11.7 MB)

Export complete (8.7s)
Results saved to /content/runs/detect/supermarket_model/weights
Predic

'runs/detect/supermarket_model/weights/best.onnx'